### Imports

In [1]:
import pandas as pd
import json
import os, sys
from tqdm import tqdm
import numpy as np
import polars as pl
from polars import col as c

sys.path.append("../")

### table creation

In [2]:
dir_path = "../data/parsed_dataframes"
file_name = "lab_records.parquet"
file_path = os.path.join(dir_path, file_name)
lab_records = pd.read_parquet(file_path)

In [3]:
for column in lab_records.columns:
    lab_records[column] = lab_records[column].replace(to_replace="nan", value=np.nan)
    lab_records[column] = pd.to_numeric(lab_records[column], errors="ignore")

/var/folders/v_/vqhfnf3s4sb7g6nfby7hc9sr0000gn/T/ipykernel_97393/1777949469.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  lab_records[column] = pd.to_numeric(lab_records[column], errors="ignore")


In [16]:
features_without_nans = (
    pd.DataFrame(
        lab_records.isna().sum(),
        columns=["count_nans"],
    )
    .sort_values("count_nans")
    .head(25)
    .index.tolist()
)

In [28]:
filtered_lab_records = (
    pl.from_pandas(lab_records)
    .select(features_without_nans)
    .with_columns(
        c("birthday").cast(pl.String).str.to_datetime("%Y%m%d").dt.date(),
        c("lab_date").cast(pl.String).str.to_datetime("%d.%m.%Y %H:%M").dt.date(),
    )
    .with_columns((c("lab_date").dt.year() - c("birthday").dt.year()).alias("age"))
    # ["Кристаллы оксалата кальция, обнаружение в моче методом компьютер-ассистированной микроскопии"]
    # .unique()
    .filter(
        c("Гемоглобин общий, массовая концентрация в крови").is_not_null(),
        c(
            "Кальций ионизированный, молярная концентрация в венозной крови"
        ).is_not_null(),
    )
    # .null_count()
    # .shape
    # .head(3)
)
filtered_lab_records.head(3)

record,gender,birthday,lab_date,"Гемоглобин общий, массовая концентрация в крови",Гематокрит крови методом автоматизированного подсчёта,"Кислород общий, молярная концентрация в венозной крови","Билирубин общий, молярная концентрация в венозной крови","Диоксид углерода, парциальное давление в венозной крови","Хлориды, молярная концентрация в венозной крови","Глюкоза, молярная концентрация в венозной крови","Натрий, молярная концентрация в венозной крови",Водородный показатель (pH) венозной крови,"Избыток оснований стандартный, расчетная молярная концентрация в венозной крови","Избыток оснований истинный, расчетная молярная концентрация в венозной крови","Карбоксигемоглобин, массовая фракция в венозной крови","Водородный показатель (pH) венозной крови, откорректированный по температуре пациента","Бикарбонат стандартный, молярная концентрация в венозной крови","Кислород, парциальное давление при 50% сатурации в венозной крови","Лактат, молярная концентрация в венозной крови","Метгемоглобин, массовая фракция в венозной крови","Насыщение гемоглобина кислородом, массовая фракция в венозной крови","Оксигемоглобин, массовая фракция в венозной крови","Калий, молярная концентрация в венозной крови","Кальций ионизированный, молярная концентрация в венозной крови",age
str,str,date,date,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""EMD_LAB_188432683_255587897""","""Мужской""",1955-11-14,2023-08-28,148.0,45.5,"""['21' '45.6' '16.1']""",13.0,30.9,113.0,5.5,136.0,7.458,-1.7,-0.8,0.7,36.6,21.6,28.72,4.2,0.6,78.3,77.3,4.0,1.0,68
"""EMD_LAB_188204627_255578735""","""Женский""",1979-07-02,2023-08-16,137.0,42.2,"""['21' '45.7' '15']""",12.0,39.9,107.0,6.2,136.0,7.399,-0.1,-0.1,1.3,36.6,24.1,27.76,2.1,1.1,79.8,77.9,3.9,0.93,44
"""EMD_LAB_187351453_255545541""","""Мужской""",1978-06-26,2023-07-02,150.0,46.0,"""['21' '25.1' '8.6']""",11.0,53.7,110.0,6.8,141.0,7.288,-0.9,-2.1,2.3,36.6,24.9,28.34,2.0,1.0,42.2,40.8,3.5,0.92,45


In [55]:
filtered_lab_records = filtered_lab_records.with_columns(
    c("Кислород общий, молярная концентрация в венозной крови").map_elements(
        lambda x: [float(number) for number in x.strip("[]'").split("' '")],
        return_dtype=list[float],
    )
)

In [56]:
filtered_lab_records.head(3)

record,gender,birthday,lab_date,"Гемоглобин общий, массовая концентрация в крови",Гематокрит крови методом автоматизированного подсчёта,"Кислород общий, молярная концентрация в венозной крови","Билирубин общий, молярная концентрация в венозной крови","Диоксид углерода, парциальное давление в венозной крови","Хлориды, молярная концентрация в венозной крови","Глюкоза, молярная концентрация в венозной крови","Натрий, молярная концентрация в венозной крови",Водородный показатель (pH) венозной крови,"Избыток оснований стандартный, расчетная молярная концентрация в венозной крови","Избыток оснований истинный, расчетная молярная концентрация в венозной крови","Карбоксигемоглобин, массовая фракция в венозной крови","Водородный показатель (pH) венозной крови, откорректированный по температуре пациента","Бикарбонат стандартный, молярная концентрация в венозной крови","Кислород, парциальное давление при 50% сатурации в венозной крови","Лактат, молярная концентрация в венозной крови","Метгемоглобин, массовая фракция в венозной крови","Насыщение гемоглобина кислородом, массовая фракция в венозной крови","Оксигемоглобин, массовая фракция в венозной крови","Калий, молярная концентрация в венозной крови","Кальций ионизированный, молярная концентрация в венозной крови",age
str,str,date,date,f64,f64,list[f64],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""EMD_LAB_188432683_255587897""","""Мужской""",1955-11-14,2023-08-28,148.0,45.5,"[21.0, 45.6, 16.1]",13.0,30.9,113.0,5.5,136.0,7.458,-1.7,-0.8,0.7,36.6,21.6,28.72,4.2,0.6,78.3,77.3,4.0,1.0,68
"""EMD_LAB_188204627_255578735""","""Женский""",1979-07-02,2023-08-16,137.0,42.2,"[21.0, 45.7, 15.0]",12.0,39.9,107.0,6.2,136.0,7.399,-0.1,-0.1,1.3,36.6,24.1,27.76,2.1,1.1,79.8,77.9,3.9,0.93,44
"""EMD_LAB_187351453_255545541""","""Мужской""",1978-06-26,2023-07-02,150.0,46.0,"[21.0, 25.1, 8.6]",11.0,53.7,110.0,6.8,141.0,7.288,-0.9,-2.1,2.3,36.6,24.9,28.34,2.0,1.0,42.2,40.8,3.5,0.92,45


In [60]:
filtered_lab_records.shape

(892, 26)

In [61]:
filtered_lab_records.null_count()

record,gender,birthday,lab_date,"Гемоглобин общий, массовая концентрация в крови",Гематокрит крови методом автоматизированного подсчёта,"Кислород общий, молярная концентрация в венозной крови","Билирубин общий, молярная концентрация в венозной крови","Диоксид углерода, парциальное давление в венозной крови","Хлориды, молярная концентрация в венозной крови","Глюкоза, молярная концентрация в венозной крови","Натрий, молярная концентрация в венозной крови",Водородный показатель (pH) венозной крови,"Избыток оснований стандартный, расчетная молярная концентрация в венозной крови","Избыток оснований истинный, расчетная молярная концентрация в венозной крови","Карбоксигемоглобин, массовая фракция в венозной крови","Водородный показатель (pH) венозной крови, откорректированный по температуре пациента","Бикарбонат стандартный, молярная концентрация в венозной крови","Кислород, парциальное давление при 50% сатурации в венозной крови","Лактат, молярная концентрация в венозной крови","Метгемоглобин, массовая фракция в венозной крови","Насыщение гемоглобина кислородом, массовая фракция в венозной крови","Оксигемоглобин, массовая фракция в венозной крови","Калий, молярная концентрация в венозной крови","Кальций ионизированный, молярная концентрация в венозной крови",age
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
